In [0]:
import torch
from torchtext import data

In [0]:
TEXT = data.Field(include_lengths = False) #tokenize -> Default: string.split
LABEL = data.LabelField(dtype = torch.float)

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train_data = data.TabularDataset('/content/gdrive/My Drive/Colab Notebooks/pytorch/nsmc/ratings_train.txt', 
                                'tsv', [('id', None), ('text', TEXT), ('label', LABEL)],
                                 skip_header = True)
test_data = data.TabularDataset('/content/gdrive/My Drive/Colab Notebooks/pytorch/nsmc/ratings_test.txt',
                                'tsv', [('id', None), ('text', TEXT), ('label', LABEL)],
                                skip_header = True)

In [5]:
print(vars(train_data[0]))
print(vars(test_data[0]))

{'text': ['아', '더빙..', '진짜', '짜증나네요', '목소리'], 'label': '0'}
{'text': ['굳', 'ㅋ'], 'label': '1'}


In [0]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(0))

In [7]:
print(len(train_data))
print(len(valid_data))

105000
45000


In [0]:
MAX_VOCAB_SIZE = 50000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.200d",
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size = BATCH_SIZE, device = device)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)

        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        return self.fc(hidden.squeeze(0))

In [0]:
class CNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, n_filters, filter_sizes,
                 output_dim, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        text = text.permute(1, 0)
        embedded = self.embedding(text)
        embedded = embedded.permute(0, 2, 1)
        conved = [F.relu(conv(embedded)) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = self.dropout(torch.cat(pooled, dim = 1))
        return self.fc(cat)

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 200
HIDDEN_DIM = 256
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
N_FILTERS = 100
FILTER_SIZES = [3,4,5]

In [0]:
model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
model = model.to(device)

In [0]:
def count_parameters(model):
     return sum(p.numel() for p in model.parameters() if p.requires_grad) 

In [16]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.8429, -0.0612,  0.7080,  ...,  1.0995, -1.0048, -0.6713],
        [ 1.1092, -0.4114, -1.3821,  ..., -0.4129,  0.2050,  0.7194],
        [ 0.2495, -0.1753,  0.7314,  ...,  0.0879,  0.9612, -0.0687],
        ...,
        [ 0.4904, -1.1527, -0.7067,  ..., -1.1262, -0.4967, -0.5861],
        [-0.6935, -0.9713, -0.5528,  ...,  0.7393, -0.3826, -0.3494],
        [-1.7043,  0.0961,  0.3996,  ...,  1.4558, -0.5707, -1.4621]],
       device='cuda:0')

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [0]:
import torch.optim as optim

In [0]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    with torch.no_grad():

        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()

        return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [23]:
for epoch in range(5):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    print(f'{epoch+1} : {train_loss}, {train_acc*100:.2f}%')

1 : 0.5777227080209745, 68.10%
2 : 0.42872876983129127, 79.06%
3 : 0.3432326131499736, 83.73%
4 : 0.2871614645696645, 86.45%
5 : 0.2458166066082711, 88.26%
